In [1]:
import numpy as np
from selenium import webdriver
from time import sleep
import random
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime

# Setting Options
options = webdriver.ChromeOptions()

# Add user agent
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
options.add_argument(f"user-agent={user_agent}")

# Other options
options.add_argument("--ignore-certificate-errors")
options.add_argument("--start-maximized")
options.add_argument("--disable-popup-blocking")
options.add_argument("--no-sandbox")

# Declare browser
driver = webdriver.Chrome(options=options)
sleep(random.randint(1,5))

# Open URL
driver.get("https://hasaki.vn/danh-muc/cham-soc-da-mat-c4.html")
sleep(random.randint(5,10))

In [8]:
# Get link, id
elems_link_id = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .block_info_item_sp')
links = [elem.get_attribute('href') for elem in elems_link_id]
product_ids = [elem.get_attribute('data-id') for elem in elems_link_id]
data_product_ids = [elem.get_attribute('data-product') for elem in elems_link_id]
sleep(random.randint(5,10))

# Get title
elems_title = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .vn_names')
title = [elem.text for elem in elems_title]
sleep(random.randint(5,10))

# Get price
elems_price = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .item_giacu')
price = [elem.text for elem in elems_price]
sleep(random.randint(5,10))

# Get discount
elems_discount = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .item_giamoi')
discount = [elem.text for elem in elems_discount]
sleep(random.randint(5,10))

In [10]:
# Get review counts:
def get_reviews_counts(string):
    start_index = string.find('(')
    end_index = string.find(')')
    review_count = string[start_index + 1:end_index]
    return review_count

elems_review_count = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .block_count_by')
review_count = [get_reviews_counts(elem.text) for elem  in elems_review_count]
sleep(random.randint(5,10))

# Get quantity_sold
elems_quantity_sold = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .item_count_by')
quantity_sold = [elem.text.strip('\n ') for elem  in elems_quantity_sold]
sleep(random.randint(5,10))

# Get product_variant
elems_product_variant = driver.find_elements(By.CSS_SELECTOR, ".ProductGrid__grid .block_info_item_sp")
product_variant = [elem.get_attribute('data-variant') for elem  in elems_product_variant]
sleep(random.randint(5,10))

In [21]:
# Read ids from crawled csv
existing_ids_df = pd.read_csv("D:/HOC KI 8/3. Graduate project/iddata_20240223_1411.csv")
existing_product_ids = existing_ids_df['product_id'].tolist()
existing_data_product_ids = existing_ids_df['data_product_id'].tolist()

# Choose new id to be crawled
new_product_ids = [id for id in product_ids if id not in existing_product_ids]
new_data_product_ids = [id for id in data_product_ids if id not in existing_data_product_ids]
# Choose details with new id
new_title = [title[i] for i, id in enumerate(data_product_ids) if id in new_data_product_ids]
new_links = [links[i] for i, id in enumerate(data_product_ids) if id in new_data_product_ids] 
new_price = [price[i] for i, id in enumerate(data_product_ids) if id in new_data_product_ids]
new_discount = [discount[i] for i, id in enumerate(data_product_ids) if id in new_data_product_ids]
new_quantity_sold = [quantity_sold[i] for i, id in enumerate(data_product_ids) if id in new_data_product_ids]
new_product_variant = [product_variant[i] for i, id in enumerate(data_product_ids) if id in new_data_product_ids]
new_review_count = [review_count[i] for i, id in enumerate(data_product_ids) if id in new_data_product_ids]

In [25]:
# Create dataframe for data crawled
product_data = pd.DataFrame(
    list(zip(new_product_ids, new_data_product_ids, new_title, new_links, new_price, 
             new_discount, new_quantity_sold, new_product_variant, new_review_count)), 
    columns = ['product_id', 'data_product_id', 'title', 'link','original_price', 
                'current_price', 'quantity_sold', 'product_variant', 'review_count'])
product_data['index']= np.arange(1, len(product_data) + 1)
sleep(random.randint(3,5))

In [26]:
# Save into csv
current_datetime = datetime.now().strftime("%Y%m%d_%H%M")
file_name = f"productdata_{current_datetime}.csv"
product_data.to_csv(f"D:/HOC KI 8/3. Graduate project/{file_name}", encoding='utf-8-sig')